In [3]:
import pandas as pd
import numpy as np
import os

In [5]:
data_dir = "Mldata/"

In [46]:
users = pd.read_csv(data_dir+"users.csv")
posts = pd.read_csv(data_dir+"posts.csv")
views = pd.read_csv(data_dir+"views.csv")

In [47]:
views.head()

,user_id,post_id,timestamp
0,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,2020-06-01T10:46:45.131Z
1,5ed3748576027d35905ccaab,5ed4cbadbd514d602c1531a6,2020-06-01T09:39:20.021Z
2,5ed0defa76027d35905cc2de,5eac305f10426255a7aa9dd3,2020-06-01T08:12:42.682Z
3,5ed0defa76027d35905cc2de,5ed1ff0276027d35905cc60d,2020-06-01T08:10:23.880Z
4,5ed0defa76027d35905cc2de,5ed3820f76027d35905ccac8,2020-06-01T08:08:54.124Z


In [48]:
views['count'] = views.groupby(['user_id','post_id']).transform('count')

In [51]:
views['max_count'] = views.groupby(['user_id'])['count'].transform('max')

In [56]:
views[views['user_id']=='5d60098a653a331687083238']

,user_id,post_id,timestamp,count,max_count
16,5d60098a653a331687083238,5ed3820f76027d35905ccac8,2020-05-31T18:01:54.308Z,1,3
32,5d60098a653a331687083238,5ed1ff0276027d35905cc60d,2020-05-31T08:21:29.911Z,1,3
109,5d60098a653a331687083238,5ecf96e876027d35905cbf46,2020-05-29T19:13:27.566Z,1,3
110,5d60098a653a331687083238,5ecfa0ca76027d35905cbf57,2020-05-29T19:10:50.010Z,1,3
111,5d60098a653a331687083238,5ed0e20776027d35905cc2fe,2020-05-29T19:10:23.887Z,1,3
...,...,...,...,...,...
1444,5d60098a653a331687083238,5e7a60edcfc8b713f5ac7d82,2020-03-26T19:03:55.284Z,1,3
1445,5d60098a653a331687083238,5e7a629ccfc8b713f5ac7d84,2020-03-26T19:01:42.384Z,1,3
1446,5d60098a653a331687083238,5e7b1e65cfc8b713f5ac7d90,2020-03-26T19:00:25.693Z,2,3
1447,5d60098a653a331687083238,5e7cc074cfc8b713f5ac7db0,2020-03-26T18:49:14.721Z,2,3


In [57]:
views['rating'] = views['count']/views['max_count']

In [58]:
views.head()

,user_id,post_id,timestamp,count,max_count,rating
0,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,2020-06-01T10:46:45.131Z,1,2,0.5
1,5ed3748576027d35905ccaab,5ed4cbadbd514d602c1531a6,2020-06-01T09:39:20.021Z,1,1,1.0
2,5ed0defa76027d35905cc2de,5eac305f10426255a7aa9dd3,2020-06-01T08:12:42.682Z,1,1,1.0
3,5ed0defa76027d35905cc2de,5ed1ff0276027d35905cc60d,2020-06-01T08:10:23.880Z,1,1,1.0
4,5ed0defa76027d35905cc2de,5ed3820f76027d35905ccac8,2020-06-01T08:08:54.124Z,1,1,1.0


In [67]:
ratings = views[['user_id','post_id','rating']]
ratings.count()

user_id    1449
post_id    1449
rating     1449
dtype: int64

In [69]:
from surprise import NormalPredictor
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import cross_validate, GridSearchCV, KFold

In [81]:
reader = Reader(rating_scale=(0, 1))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(ratings, reader)

In [82]:
algo = SVD()

trainset = data.build_full_trainset()
algo.fit(trainset)

testset = trainset.build_testset()
predictions = algo.test(testset)
# RMSE should be low as we are biased
accuracy.rmse(predictions, verbose=True)

RMSE: 0.1083


0.10832567263232866

In [83]:
print('Grid Search...')
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005]}
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)
grid_search.fit(data)

Grid Search...


In [84]:
algo = grid_search.best_estimator['rmse']

# retrain on the whole set A
trainset = data.build_full_trainset()
algo.fit(trainset)

# Compute biased accuracy on A
predictions = algo.test(trainset.build_testset())
print('Biased error on A,', end='   ')
accuracy.rmse(predictions)

Biased error on A,   RMSE: 0.1501


0.15014602208925218

In [114]:
def get_rating(model, userID, itemID, actual=0):
    print("Predicting rating of \nuserID: {} \nitemID: {}\n".format(userID, itemID))
    
    if actual==0:
        rating = model.predict(uid=userID, iid=itemID).est
        print("Rating: {}".format(rating))
    else:
        pred = model.predict(uid=userID, iid=itemID, r_ui=actual)
        rating = pred.est
        print("Rating: {}\tActual: {}".format(rating, actual))
    
    return rating

In [ ]:
def get_rating(model, userID, itemID, actual=0):
    print("Predicting rating of \nuserID: {} \nitemID: {}\n".format(userID, itemID))
    
    if actual==0:
        rating = model.predict(uid=userID, iid=itemID).est
        print("Rating: {}".format(rating))
    else:
        pred = model.predict(uid=userID, iid=itemID, r_ui=actual)
        rating = pred.est
        print("Rating: {}\tActual: {}".format(rating, actual))
    
    return rating

In [115]:
userID = '5df49b32cc709107827fb3c7'
itemID = '5ec821ddec493f4a2655889e'

get_rating(algo, userID,itemID, 0.5)

Predicting rating of 
userID: 5df49b32cc709107827fb3c7 
itemID: 5ec821ddec493f4a2655889e

Rating: 0.6942829458973089	Actual: 0.5


0.6942829458973089

In [150]:
def get_rating(model, userID, items, num):
  
    items = items.to_frame(name='item_id')
    items['rating'] = items['item_id'].apply(lambda x: model.predict(uid=userID, iid=x).est)
    blog_indices = items.sort_values('rating', ascending=False)['item_id'].unique()[:num]
    print(blog_indices)
    return posts[posts['_id'].isin(blog_indices)]['title'] 

In [151]:
userID = '5df49b32cc709107827fb3c7'
itemID = '5ec821ddec493f4a2655889e'

get_rating(algo, userID,ratings['post_id'], 10)

['5eb2c11210426255a7aaa052' '5e847970a3258347b42f245e'
 '5e9b0475a3258347b42f2c6b' '5ed2502b76027d35905cc7db'
 '5e901e9da3258347b42f2711' '5e895ceca3258347b42f25b6'
 '5e948155a3258347b42f2850' '5d62abaa65218653a132c956'
 '5e947c4aa3258347b42f283e' '5ed1ff0276027d35905cc60d']


0                                            hello there
213       8 Free Data Science Courses & Tutorials [2020]
222            What are Set Off and Carry Forward Losses
235    Advantages of Online Shopping Over Offline Sho...
248                          IMPORTANCE OF MENTAL HEALTH
250       HOW TO PREPARE YOURSELF FOR A BETTER TOMORROW?
316                                Not too much to lose?
395                                       Gangster Style
470       AWS services and how to launch OS on AWS Cloud
486                                          Learning...
Name: title, dtype: object